In [1]:
from enum import Enum
from typing import Literal

from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent



In [2]:

# -----------------------------
# 1) Tool tipada con Pydantic
# -----------------------------
class Operation(str, Enum):
    add = "add"
    sub = "sub"
    mul = "mul"
    div = "div"


class CalcInput(BaseModel):
    a: float = Field(..., description="Primer número")
    b: float = Field(..., description="Segundo número")
    op: Operation = Field(..., description="Operación: add|sub|mul|div")


@tool(args_schema=CalcInput)
def calculate(a: float, b: float, op: Operation) -> float:
    """Realiza una operación aritmética entre dos números y devuelve el resultado."""
    if op == Operation.add:
        return a + b
    if op == Operation.sub:
        return a - b
    if op == Operation.mul:
        return a * b
    if op == Operation.div:
        if b == 0:
            raise ValueError("División por cero no permitida.")
        return a / b
    raise ValueError(f"Operación no soportada: {op}")



In [3]:
# -----------------------------
# 2) Agente: decide usar tool o no
# -----------------------------
def build_agent():
    llm = ChatOpenAI(model="gpt-5", temperature=0)
    tools = [calculate]

    system_prompt = (
        "Eres un agente en español.\n"
        "Tienes disponible una herramienta llamada `calculate`.\n"
        "Regla:\n"
        "- Usa `calculate` SOLO cuando haya un cálculo explícito o cuando sea útil para evitar errores.\n"
        "- Si la pregunta es conceptual (sin cálculo), responde sin herramientas.\n"
        "Cuando uses la herramienta, interpreta el resultado y explica brevemente."
    )

    # En LangChain 1.2, create_agent recibe system_prompt (no prompt)
    agent = create_agent(
        model=llm,
        tools=tools,
        system_prompt=system_prompt,
        debug=True,  # <-- clave: muestra en consola tool calls y mensajes internos del runtime
    )
    return agent


def run_query(agent, text: str):
    result = agent.invoke({"messages": [HumanMessage(content=text)]})
    # El último mensaje suele ser la respuesta final al usuario
    last = result["messages"][-1]
    return getattr(last, "content", last)




In [4]:
agent = build_agent()
print("Agente creado. Probando con diferentes preguntas...")
# Caso 1: debería usar la tool
q1 = "Calcula 12.5 dividido entre 2.5. Usa la herramienta."
print("\n=== Pregunta 1 (debería usar tool) ===")
print(run_query(agent, q1))

# Caso 2: NO debería usar la tool (pregunta conceptual)
q2 = "¿Qué diferencia hay entre una tool y una API? Responde sin hacer cálculos."
print("\n=== Pregunta 2 (no debería usar tool) ===")
print(run_query(agent, q2))

# Caso 3: el agente puede elegir tool por seguridad/precisión
q3 = "¿Cuánto es 19 * 37? No adivines: asegúrate del resultado."
print("\n=== Pregunta 3 (probablemente use tool) ===")
print(run_query(agent, q3))


Agente creado. Probando con diferentes preguntas...

=== Pregunta 1 (debería usar tool) ===
[values] {'messages': [HumanMessage(content='Calcula 12.5 dividido entre 2.5. Usa la herramienta.', additional_kwargs={}, response_metadata={}, id='4993749a-ceb5-45af-bdfc-683ca6b178dc')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 162, 'prompt_tokens': 257, 'total_tokens': 419, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8TuDzXhvKNpB5tCU3FYiYDZr3mUX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c52b2-74a2-7162-ab48-ac96c3a8fd16-0', tool_calls=[{'name': 'calculate', 'args': {'a'